In [2]:
%pylab inline
from astropy.io import ascii


Populating the interactive namespace from numpy and matplotlib


In [3]:
tab = ascii.read("SFH_nonparametric.csv")
tab

age_5559,SFR_5559,SFR_5559_lower,SFR_5559_upper,age_2768,SFR_2768,SFR_2768_lower,SFR_2768_upper,age_4589,SFR_4589,SFR_4589_lower,SFR_4589_upper,age_2272,SFR_2272,SFR_2272_lower,SFR_2272_upper
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
7.0,2.805301469,1.924388032,3.870422334,7.0,0.026871016,0.020519688,0.032353934,7.0,0.019028994,0.004420896,0.032284825,7.0,0.020797865,0.011647041,0.028812223
7.4772,2.805301469,1.924388032,3.870422334,7.4772,0.026871016,0.020519688,0.032353934,7.4772,0.019028994,0.004420896,0.032284825,7.4772,0.020797865,0.011647041,0.028812223
8.0,2.557045217,1.566949456,4.013154327,8.0,0.024268191,0.013056999,0.044073827,8.0,0.023904186,0.007786247,0.069333359,8.0,0.020770629,0.010636885,0.036934683
8.41182821,1.829041725,1.047394451,3.247636364,8.41295485,0.025780505,0.010468289,0.055395873,8.41182821,0.043505253,0.011607563,0.166872281,8.41272558,0.022753317,0.009018913,0.054321541
8.82365641,1.606295216,0.958543437,2.739943847,8.82590969,0.029062501,0.010392243,0.074774415,8.82365641,0.124691374,0.021557167,0.70851965,8.82545117,0.027848958,0.009408316,0.067706756
9.23548462,2.169113737,1.241045789,3.601944396,9.23886454,0.034377901,0.010790162,0.118251227,9.23548462,1.174755176,0.069269332,3.829472711,9.23817675,0.039456334,0.01044601,0.112032807
9.64731283,4.102752402,2.652850268,5.694333511,9.65181939,0.057471375,0.012155218,0.219240401,9.64731283,10.45083918,5.830641093,15.984388,9.65090233,0.445498287,0.06453332,2.166796121
10.05914103,6.561043336,5.18591365,7.842313275,10.06477424,16.13372962,13.53755454,18.67769776,10.05914103,10.32569154,8.223229878,13.23611734,10.06362792,9.684444097,8.208922307,10.97527874
10.12972211,6.239822973,3.210173111,10.04703374,10.13535531,29.41057099,16.92027474,41.95219126,10.12972211,8.180266377,3.750407111,15.70819277,10.13420899,8.213800018,3.657780647,14.68750166


### Defining functions and reading in data

In [7]:
import scipy.interpolate as interp
import scipy.integrate as integ

def dtd_cc(t_Myr):
    #Delay-time distribution for core-collapse supernovae from Zapartas et al 2017
    #Case of single stars, Eq A1 in paper
    #The 1.0e6 prefactor is so that units are in ccSN/Myr/Msun, which helps the time integration of the DTD
    #to come out in units of ccSN/Msun. 
    return 1.0e6*np.piecewise(t_Myr, [t_Myr<3., (t_Myr>=3.) & (t_Myr<25.), (t_Myr>=25.) & (t_Myr<=48),t_Myr>48],\
                     [0, lambda t: (1.0e-9)*(-2.83 + (8.7*np.log10(t)) - 2.07*(np.log10(t)**2))*(1/t), \
                      lambda t: (1.0e-8)*(-4.85 + (6.55*np.log10(t)) - 1.92*(np.log10(t)**2))*(1/t), \
                      0])

def dtd_cc_bin(t_Myr):
    #Zapartas et al DTD, but for binaries (Eq A2 in paper)
    return 1.0e6*np.piecewise(t_Myr, [t_Myr<=3., (t_Myr>=3.) & (t_Myr<25.), (t_Myr>=25.) & (t_Myr<=48),\
                      (t_Myr>=48.) & (t_Myr<=200.),t_Myr>=200.],\
                     [0, lambda t: (1.0e-9)*(-2.65 + (7.51*np.log10(t)) - 0.98*(np.log10(t)**2))*(1/t), \
                      lambda t: (1.0e-8)*(-0.89 + (1.73*np.log10(t)) - 0.51*(np.log10(t)**2))*(1/t), \
                      lambda t: (1.0e-8)*(3.46 - (2.98*np.log10(t)) + 0.65*(np.log10(t)**2))*(1/t),
                      0])


def dtd_ia(t_Myr):
    # Type Ia SN DTD
    #Eq 13, Maoz & Mannucci 2012, PASA 2019 447
    return 1.0e6*np.piecewise(t_Myr, [t_Myr<40., (t_Myr>=40.) & (t_Myr<=100.0e4), t_Myr>100.0e4], \
                     [0, lambda t: (4.0e-10)*(t)**(-1.0), 0])

For each galaxy's star-formation history ($\dot{M}(t)$, in units $M_{\odot}$/yr), you can calculate the SN rate $R_{SN}$ as in Eq 10 of [Maoz & Badenes 2010](https://arxiv.org/abs/1003.3031) 

\begin{equation}
R_{SN} = \int \dot{M}(t-\tau) \Psi(\tau) d\tau
\end{equation}

For each age bin (e.g. $t_{lo,1}-t_{up,1}, t_{lo,2}-t_{up,2}, t_{lo,3}-t_{up,3}, ...$) and corresponding star-formation rates in the bin ($\dot{M}_1$, $\dot{M}_2$, $\dot{M}_3$,...) we can spread this integral out as 
\begin{equation}
R_{SN} = \int_{t_{lo,1}}^{t_{up,1}} \dot{M}_1(t-\tau) \Psi(\tau) d\tau + \int_{t_{lo,2}}^{t_{up,2}} \dot{M}_2(t-\tau) \Psi(\tau) d\tau + \int_{t_{lo,3}}^{t_{up,3}} \dot{M}_3(t-\tau) \Psi(\tau) d\tau + ...
\end{equation}

Since the SFR in each age bin is a constant, we just take it out of the integral and calculate the integral over the DTD
\begin{equation}
R_{SN} = \dot{M}_1 \int_{t_{lo,1}}^{t_{up,1}} \Psi(\tau) d\tau + \dot{M}_2 \int_{t_{lo,2}}^{t_{up,2}} \Psi(\tau) d\tau + \dot{M}_3 \int_{t_{lo,3}}^{t_{up,3}} \Psi(\tau) d\tau + ...
\end{equation}

(Note: I think you can get away with just evaluating the DTD at the central age bin value (i.e. $\int_{t_{lo,1}}^{t_{up,1}} \Psi(\tau) d\tau \approx \Psi_1 (t_{up,1}-t_{lo,1})$). Can check this later.)

In [10]:
galaxy = ["5559","2768","4589","2272"]
for g in galaxy:
    #Reading age limits, sfrs  from Vic's star-formation histories
    logagelims = np.concatenate(([0], tab["age_{}".format(g)].data)) #Adding 0 to the first bin
    sfr = tab["SFR_{}".format(g)].data
    sfr_low = tab["SFR_{}_lower".format(g)].data
    sfr_up = tab["SFR_{}_upper".format(g)].data
    ages = 10**logagelims

    #integral over the DTD for CC
    intDTD_cc = np.array([integ.quad(dtd_cc_bin, ages[j]/1.0e6, ages[j+1]/1.0e6)[0] for j in range(len(ages[:-1]))])
    #integral over the DTD for Ia
    intDTD_ia = np.array([integ.quad(dtd_ia, ages[j]/1.0e6, ages[j+1]/1.0e6)[0] for j in range(len(ages[:-1]))])
    #SN rates = SFR*int(DTD)
    Rsn_cc = np.dot(sfr.T, intDTD_cc)
    Rsn_ia = np.dot(sfr.T, intDTD_ia)

    print("Rates in SN/yr")
    print("Core collapse Rate for {}: ".format("NGC"+g), Rsn_cc)
    print("Thermonuclear Rate for {}: ".format("NGC"+g), Rsn_ia)
    print("Ia/CC Fraction for {}: ".format("NGC"+g), Rsn_ia/Rsn_cc)
    print("\n")


Rates in SN/yr
Core collapse Rate for NGC5559:  0.034406841182838384
Thermonuclear Rate for NGC5559:  0.007513507685876254
Ia/CC Fraction for NGC5559:  0.21837249301525183


Rates in SN/yr
Core collapse Rate for NGC2768:  0.0003322446498970088
Thermonuclear Rate for NGC2768:  0.008112989144890429
Ia/CC Fraction for NGC2768:  24.418720203336132


Rates in SN/yr
Core collapse Rate for NGC4589:  0.00027315350875378914
Thermonuclear Rate for NGC4589:  0.008930632707434731
Ia/CC Fraction for NGC4589:  32.69455606914603


Rates in SN/yr
Core collapse Rate for NGC2272:  0.0002663352225021736
Thermonuclear Rate for NGC2272:  0.004426545026648021
Ia/CC Fraction for NGC2272:  16.620201357752805


